In [31]:
# Setting up required libraries
import pandas as pd
import numpy as np
import requests
import lxml
import cchardet
import re
import concurrent.futures
from bs4 import BeautifulSoup, SoupStrainer
from multiprocessing import Pool
from multiprocessing import cpu_count

# Function Definitions:
Two main functions, get_results old and new. These correspond to the two website styles

In [32]:
def get_results_new(url, sex, year):
    #Function to scrape modern virgin london marathon results page (2020 and 2019)
    #Set up empty dataframe for results
    results = pd.DataFrame()
    
    #Use requests to get content from site
    site=requests.get(url).content
    #Soup strainer restricts content to sped up soup
    strainer = SoupStrainer(class_="section-main")
    #Parse the html
    soup = BeautifulSoup(site,'lxml', parse_only=strainer)
    #fields = soup.find(class_='section-main')

    #Loop through each row and column to create a list of cells
    my_table = []
    for row in soup.find_all(class_='list-group-item'):
        row_data = []
        for cell in row.find_all(class_='list-field'):
            row_data.append(cell.text)
        
        #If the row isn't empty, then create a dict of the row to create datafram from
        if(len(row_data) > 0):
            data_item = {"Place (Overall)": row_data[0],
                        "Place (Gender)": row_data[1],
                        "Place (Category)": row_data[2],
                        "Name": row_data[3],
                        "Sex": sex,
                        "Club": row_data[4],
                        "Running Number": row_data[5],
                        "Category": row_data[6],
                        "Finish": row_data[8],
                        "Year": year
            }
            my_table.append(data_item)
    
    #Strip table header
    df = pd.DataFrame(my_table).iloc[1:]
        
    #Append to results
    results = results.append(df)
    
    return results

In [33]:
def get_results_old(url, sex, year):
    #Function to scrape old virgin london marathon results page (2014 to 2018)
    #Set up empty dataframe for results
    results = pd.DataFrame()

    #Use requests to get content from site
    site=requests.get(url).content
    #Soup strainer restricts content to sped up soup
    strainer = SoupStrainer('tbody')
    #Parse the html
    soup = BeautifulSoup(site,'lxml', parse_only=strainer)

    my_table = []
    for row in soup.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            #Check if cell has alt text, if so use that as data
            alt_text = cell.find('span')
            if alt_text != None:
                cell = alt_text['title']
            else:
                cell = cell.text
            row_data.append(cell)
            
        #If the row isn't empty, then create a dict of the row to create datafram from
        if(len(row_data) > 0 and year != 2014):
            data_item = {"Place (Overall)": row_data[0],
                            "Place (Gender)": row_data[1],
                            "Place (Category)": row_data[2],
                            "Name": row_data[3],
                            "Sex": sex,
                            "Club": row_data[4],
                            "Running Number": row_data[5],
                            "Category": row_data[6],
                            "Finish": row_data[8],
                            "Year": year
            }
            my_table.append(data_item)
        elif(len(row_data) > 0 and year == 2014):
            data_item = {"Place (Overall)": row_data[0],
                            "Place (Gender)": row_data[1],
                            "Place (Category)": row_data[2],
                            "Name": row_data[3],
                            "Sex": sex,
                            "Club": row_data[5],
                            "Running Number": row_data[6],
                            "Category": row_data[7],
                            "Finish": row_data[9],
                            "Year": year
            }
            my_table.append(data_item)

    #Strip table header
    df = pd.DataFrame(my_table)

    #Append to results
    results = results.append(df)

    return results

In [34]:
def get_results(url):
    #Function choose what results func to apply
    #Used to allow single function for pool.map
    #Check what year the url is
    year = int(re.search('\.com\/(\d{4})\/', url).group(1))
    sex = re.search('sex%5D=(\w)', url).group(1)
    if year >= 2019:
        data = get_results_new(url, sex, year)
    elif year >= 2010:
        data = get_results_old(url, sex, year)
    else:
        data = None
    return data

In [35]:
def get_virgin_urls(sex, pages, year):
    # Get a list of urls, this is needed to be used to apply function to to then use multiprocessing
    urls = ['NaN'] * pages
    if year >= 2019:
        for i in range(len(urls)):
            urls[i] =  'https://results.virginmoneylondonmarathon.com/' \
                        +str(year) \
                        +'/?page=' \
                        +str(i+1) \
                        +'&event=ALL&num_results=1000&pid=search&pidp=results_nav&search%5Bsex%5D=' \
                        +sex \
                        +'&search%5Bage_class%5D=%25&search%5Bnation%5D=%25&search_sort=name'


    elif year >= 2014:
        for i in range(len(urls)):
            urls[i] = ('https://results.virginmoneylondonmarathon.com/'
                        +str(year)
                        +'/?page='
                        +str(i+1)
                        +'&event=MAS&num_results=1000&pid=list&search%5Bage_class%5D=%25&search%5Bsex%5D='
                        +sex)

    elif year >= 2010:
        for i in range(len(urls)):
            urls[i] = ('https://results.virginmoneylondonmarathon.com/'
                        +str(year)
                        +'/index.php?page='
                        +str(i+1)
                        +'&event=MAS&num_results=1000&pid=search&search%5Bsex%5D='
                        +sex)

    return urls

# Time to get to work

Use get_virgin_urls with a list of page numbers (need to create loop for that) and a range of years to produce the list of urls that we will iterate over

NOTE: Is possible to scrape 2010, but slightly diff. format so need to produce a different function for that.

In [36]:
urls = []
#Get no. of pages using technique like
#Not kept in/included in functions because requests take forever!
# site_m=requests.get(url1+'1'+url2+'M').text
# site_w=requests.get(url1+'1'+url2+'W').text
# soup_m = BeautifulSoup(site_m,'lxml')
# soup_w = BeautifulSoup(site_w,'lxml')

# m_pages = int(soup_m.find(class_='pages').text[-4:-2])
# w_pages = int(soup_w.find(class_='pages').text[-4:-2])
# print(m_pages, w_pages)
pages_men = [24, 23, 25, 23, 24, 24, 24, 24, 25, 22]
pages_women = [14, 13, 13, 14, 15, 16, 16, 17, 18, 22]
for i, year in enumerate(range(2011, 2021)):
    w_urls = get_virgin_urls('W', pages_women[i], year)
    m_urls = get_virgin_urls('M', pages_men[i], year)
    new_urls = m_urls + w_urls
    urls = urls + new_urls

In [47]:
urls = []
pages_men = [3] * 10
pages_women = [3] * 10
for i, year in enumerate(range(2011, 2020)):
    w_urls = get_virgin_urls('W', pages_women[i], year)
    m_urls = get_virgin_urls('M', pages_men[i], year)
    new_urls = m_urls + w_urls
    urls = urls + new_urls

The following cell uses ```multiprocess.pool``` to divide the work of making requests and parsing between a number of worker processes. This currently doesn't lead to any appreciable improvement in speed, needs further investigation! Possibly need to investigate proper threading.

This process requires an iterable and a function to apply it over.

In [37]:
# %%timeit -n 1 -r 1
# #Setup multiprocessing and start scraping!
# pool = Pool(8)
# #Scrape multiprocessing
# data = pool.map(get_results, urls)
# #Cleanup after yourself
# pool.terminate()
# pool.join()

The following cell does a similar process, but using multithreading instead of multiprocessing.

In [48]:
#Trying using multithreading instead of multiprocessing
MAX_THREADS = 30
threads = min(MAX_THREADS, len(urls))

with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    data = list(executor.map(get_results, urls))

In [24]:
#Get dataframe from list of df (sep cell to allow for recreation without re-parsing)
results = pd.concat(data)

In [25]:
#Some data cleaning
#Remove leftover titles
results['Club'] = results['Club'].str.replace("Club", "", regex=False)
results['Running Number'] = results['Running Number'].str.replace("Running Number", "", regex=False)
results['Running Number'] = results['Running Number'].str.replace("Runner Number", "", regex=False)
results['Category'] = results['Category'].str.replace("Category", "", regex=False)
results['Finish'] = results['Finish'].str.replace("Finish", "", regex=False)

#Extract country groups, like (USA), from Name group
results['Country'] = results['Name'].str.extract(r'(\([A-Z]{3,}\))')
#Remove brackets in country
results['Country'] = results['Country'].str.replace(r'\(|\)', "")
#Remove country group from name column
results['Name'] = results['Name'].str.replace(r'(\([A-Z]{3}\))', "")

#Split first/lastname into new columns
results['Name'] = results['Name'].str.replace(r'(»)', "")
LastFirst = results['Name'].str.split(pat=",", n=1, expand=True) 
results['FirstName'], results['LastName'] = LastFirst[1], LastFirst[0]
#Remove comma from Name column, so that this can be saved as a CSV ----- Must happen after splitting Name into two cols!!
results['Name'] = results['Name'].str.replace(r'(\,)', "")
#Replace non-standard '–' with NaN for missing vals
results = results.replace('–', np.nan)
results = results.replace('', np.nan)

In [28]:
results =  results.astype({"Place (Overall)": 'Int64',
                           "Place (Gender)": 'Int64',
                           "Place (Category)": 'Int64',
                           "Name": str,
                           "Sex": str,
                           "Club": str,
                           "Running Number": 'Int64',
                           "Category": str,
                           "Year": 'Int64'})
results['Finish'] = pd.to_timedelta(results['Finish'])

In [29]:
# Let's see what we've got
results

,Place (Overall),Place (Gender),Place (Category),Name,Sex,Club,Running Number,Category,Finish,Year,Country,FirstName,LastName
0,1,1,1,Mutai Emmanuel,M,nan,8,18-39,0 days 02:04:40,2011,KEN,Emmanuel,Mutai
1,2,2,2,Lel Martin,M,nan,27,18-39,0 days 02:05:45,2011,KEN,Martin,Lel
2,3,3,3,Makau Patrick,M,nan,3,18-39,0 days 02:05:45,2011,KEN,Patrick,Makau
3,4,4,4,Dos Santos Marlison Gomes,M,nan,13,18-39,0 days 02:06:34,2011,BRA,Marlison Gomes,Dos Santos
4,5,5,5,Kebede Tsegaye,M,nan,1,18-39,0 days 02:07:48,2011,ETH,Tsegaye,Kebede
...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,21445,6469,242,Barrow Susan,W,Rutland Running & Triathlon,23582,55-59,0 days 04:25:20,2019,GBR,Susan,Barrow
997,42539,17767,2943,Barrow Tracy,W,nan,12302,40-44,0 days 08:18:53,2019,GBR,Tracy,Barrow
998,10669,2574,3,Barrow-Green June,W,Victoria Park Harriers and Tower Hamlets AC,33934,65-69,0 days 03:46:34,2019,GBR,June,Barrow-Green
999,11000,2676,542,Barrow-Williams Samantha,W,nan,48131,40-44,0 days 03:47:55,2019,GBR,Samantha,Barrow-Williams


In [30]:
# And quickly save them in a csv
results.to_csv(r'C:\Users\michael.walshe\Documents\Python Projects\scrape_london_marathon\London_Marathon_Big.csv', index=False, header=True)

In [ ]:
#Profiling function to find bottlenecks, need to speed up parser more???
# url = 'https://results.virginmoneylondonmarathon.com/2019/?page=1&event=ALL&'+ \
#       'num_results=1000&pid=search&pidp=results_nav&search%5Bsex%5D=M&search%5Bage_'+ \
#        'class%5D=%25&search%5Bnation%5D=%25&search_sort=name'

# %lprun -f get_results_new get_results_new(url, "M", 2019)